In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, KFold
import random

from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.feature_extraction.text import TfidfVectorizer

from keras.preprocessing import text, sequence
from keras.layers import Embedding, SpatialDropout1D
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Input
from keras.optimizers import RMSprop
import keras.backend as K
from keras.layers import Dense, Input, GRU, LSTM, Bidirectional, Dropout, CuDNNLSTM, CuDNNGRU, GlobalAveragePooling1D, GlobalMaxPool1D
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.engine.topology import Layer, InputSpec
from keras import initializers as initializers, regularizers, constraints

from numpy.random import seed
from tensorflow import set_random_seed
import random as rn
import os

In [19]:
# GOOGLE COLAB SETUP

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#2. Get the file
data_path          = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/data'
codes_path         = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/codes'
cv_models_path     = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/cv_models'
models_path        = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/models'


#3. Read file as panda dataframe
train         = pd.read_csv(f'{data_path}/train_cleaned_no_punkt.csv') 
test_labelled = pd.read_csv(f'{data_path}/test_labelled_cleaned_no_punkt.csv') 
test_unlabelled = pd.read_csv(f'{data_path}/test_unlabelled_cleaned_no_punkt.csv') 

In [0]:
train['mal']         = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
train.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
train.comment_text.fillna("empty", inplace=True)

test_labelled['mal'] = test_labelled[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
test_labelled.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
test_labelled.comment_text.fillna("empty", inplace=True)

test_unlabelled.comment_text.fillna("empty", inplace=True)

# CHANGE TRAIN AND TEST, MIX TO GET SIMILAR DISTRIBUTION
from sklearn.model_selection import train_test_split
rs=42
X_train1, X_test1, y_train1, y_test1  = train_test_split(train.drop('mal', axis=1), train.mal, stratify=train.mal, test_size=0.29, random_state=rs )
X_train2, X_test2, y_train2, y_test2  = train_test_split(test_labelled.drop('mal', axis=1), test_labelled.mal, stratify=test_labelled.mal, test_size=0.29, random_state=rs)

X = np.concatenate((X_train1.comment_text, X_train2.comment_text))
y = np.concatenate((y_train1, y_train2))

X_test = np.concatenate((X_test1.comment_text, X_test2.comment_text))
y_test = np.concatenate((y_test1, y_test2))

X_unlab = np.array(test_labelled.comment_text)

In [0]:
#X = train.comment_text
#y = train.mal

max_features = 40000
maxlen       = 400
dropout_rate = 0.25
rs           = 42
epochs       = 4
batch_size   = 256
embed_dim    = 50
rec_units    = 150


seed(rs)
set_random_seed(rs)
rn.seed(rs)

os.environ['PYTHONHASHSEED']=str(rs)

In [0]:
def gru_keras(max_features, maxlen, dropout_rate, embed_dim, rec_units):
    if K.backend == 'tensorflow':        
        K.clear_session()
    input_layer = Input(shape=(maxlen,))
    embedding_layer = Embedding(max_features, output_dim=embed_dim, trainable=True)(input_layer)
    x = SpatialDropout1D(dropout_rate)(embedding_layer)
    x = Bidirectional(CuDNNGRU(units=rec_units, return_sequences=False))(x)
    #if reduction == 'average':
    #  x = GlobalAveragePooling1D()(x)
    #elif reduction == 'maximum':
    #  x = GlobalMaxPool1D()(x)
    #elif reduction == 'attention':
    #  x = AttentionWithContext()(x)
      
    output_layer = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(loss='binary_crossentropy',
                  optimizer=RMSprop(clipvalue=1, clipnorm=1),
                  metrics=['acc'])
    #print( model.summary())
    return model

In [0]:
kf = StratifiedKFold(n_splits=5, random_state=rs)
auc = []
roc = []
c = 0

### Average Pooling

### PSEUDO-LABELLING

In [0]:
not_trained=True
if not_trained:
  ps_labels = pd.DataFrame({'text'   : X_unlab, 
                            'model_1': np.zeros(len(X_unlab)), 
                            'model_2': np.zeros(len(X_unlab)), 
                            'model_3': np.zeros(len(X_unlab)),
                            'model_4': np.zeros(len(X_unlab)), 
                            'model_5': np.zeros(len(X_unlab))})

  for c, (train_index, val_index) in enumerate(kf.split(X, y)):
      print(f' fold {c}')

      X_train      = X[train_index]
      y_train      = y[train_index]

      tokenizer = text.Tokenizer(num_words=max_features)
      tokenizer.fit_on_texts(X_train)     
      
      list_tokenized_train = tokenizer.texts_to_sequences(X_train)
      list_tokenized_unlab = tokenizer.texts_to_sequences(X_unlab)

      X_train              = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
      X_unlab_              = sequence.pad_sequences(list_tokenized_unlab, maxlen=maxlen)

      model                = gru_keras(max_features, maxlen, dropout_rate, embed_dim, rec_units)
      history              = model.fit(X_train, y_train, batch_size=batch_size, epochs=4, shuffle=True, verbose=1)
      probs_unlab          = model.predict(X_unlab_, batch_size=batch_size, verbose=1)

      ps_labels.loc[:, [f'model_{c+1}']] = probs_unlab
      K.clear_session()
      del model
      
      
  ps_labels['mean_score'] = ps_labels.drop('text', axis=1).mean(axis=1)
  ps_labels['mean_score_without_5fold'] = ps_labels.drop(['text', 'mean_score', 'model_5'], axis=1).mean(axis=1)
  ps_labels.to_csv(f'{data_path}/pseudo_labels_scores.csv', index=False)

 fold 0
Epoch 1/4
126974/126974 [==============================] - 126s 991us/step - loss: 0.1840 - acc: 0.9365
Epoch 2/4
126974/126974 [==============================] - 125s 987us/step - loss: 0.1347 - acc: 0.9479
Epoch 3/4
126974/126974 [==============================] - 125s 985us/step - loss: 0.1158 - acc: 0.9553
Epoch 4/4
63978/63978 [==============================] - 20s 306us/step
 fold 1
Epoch 1/4
126974/126974 [==============================] - 126s 989us/step - loss: 0.1833 - acc: 0.9352
Epoch 2/4
126974/126974 [==============================] - 125s 986us/step - loss: 0.1249 - acc: 0.9522
Epoch 3/4
126974/126974 [==============================] - 125s 985us/step - loss: 0.1139 - acc: 0.9554
Epoch 4/4
63978/63978 [==============================] - 20s 308us/step
 fold 2
Epoch 1/4
126976/126976 [==============================] - 124s 975us/step - loss: 0.1809 - acc: 0.9366
Epoch 2/4
126976/126976 [==============================] - 124s 979us/step - loss: 0.1232 - acc: 0.9532


In [0]:
ps_labels = pd.read_csv(f'{data_path}/pseudo_labels_scores.csv')
ps_labels['mean_score_without_5fold'] = ps_labels.drop(['text', 'mean_score', 'model_5'], axis=1).mean(axis=1)
ps_labels['y'] = 1
ps_labels.loc[ps_labels['mean_score_without_5fold']< 0.1, 'y'] = 0

In [0]:
#X = pd.concat([pd.Series(X), ps_labels.loc[ps_labels['mean_score_without_5fold']>0.6, 'text']], axis=0)
#y = pd.concat([pd.Series(y), ps_labels.loc[ps_labels['mean_score_without_5fold']>0.6, 'y']], axis=0)

#X = pd.concat([pd.Series(X), ps_labels.loc[ps_labels['mean_score_without_5fold']< 0.1 , 'text']], axis=0)
#y = pd.concat([pd.Series(y), ps_labels.loc[ps_labels['mean_score_without_5fold']< 0.1, 'y']], axis=0)

In [0]:
#X = np.array(X)
#y = np.array(y)

In [0]:
for train_index, val_index in kf.split(X, y):
    print(f' fold {c}')
    X_train, X_val       = X[train_index], X[val_index]
    y_train, y_val       = y[train_index], y[val_index] 
    
    X_train = pd.concat([pd.Series(X_train), ps_labels.loc[ps_labels['mean_score_without_5fold']>0.6, 'text']], axis=0)
    y_train = pd.concat([pd.Series(y_train), ps_labels.loc[ps_labels['mean_score_without_5fold']>0.6, 'y']], axis=0)
    
    X_train = np.array(X)
    y_train = np.array(y)
    
    tokenizer = text.Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(X_train)
    list_tokenized_train = tokenizer.texts_to_sequences(X_train)
    list_tokenized_val   = tokenizer.texts_to_sequences(X_val)
    X_train              = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
    X_val                = sequence.pad_sequences(list_tokenized_val, maxlen=maxlen)
    model                = gru_keras(max_features, maxlen, dropout_rate, embed_dim, rec_units)
    print('Fitting')
    history              = model.fit(X_train, y_train, batch_size=batch_size, validation_data=(X_val, y_val), epochs=4, shuffle=True, verbose=1)
    probs                = model.predict(X_val, batch_size=batch_size, verbose=1)
    
    model.save_weights(f'{cv_models_path}/BGRU_avpool_SEMI_fold_06_onlyTrain_{c}.h5')
    
    auc_f                = average_precision_score(y_val, probs)
    auc.append(auc_f)
    roc_f                = roc_auc_score(y_val, probs)
    roc.append(roc_f)
    print(f' average precision {auc_f}')
    print(f' roc auc {roc_f}')
    c += 1
    del model
    K.clear_session()

 fold 4
Fitting
Train on 158719 samples, validate on 31745 samples
Epoch 1/4
158719/158719 [==============================] - 166s 1ms/step - loss: 0.1720 - acc: 0.9399 - val_loss: 0.1198 - val_acc: 0.9580
Epoch 2/4
158719/158719 [==============================] - 165s 1ms/step - loss: 0.1199 - acc: 0.9545 - val_loss: 0.0964 - val_acc: 0.9628
Epoch 3/4
158719/158719 [==============================] - 165s 1ms/step - loss: 0.1115 - acc: 0.9567 - val_loss: 0.1005 - val_acc: 0.9606
Epoch 4/4
31745/31745 [==============================] - 10s 313us/step
 average precision 0.9178970210368779
 roc auc 0.9839413825573682
 fold 5
Fitting
Train on 158719 samples, validate on 31745 samples
Epoch 1/4
158719/158719 [==============================] - 166s 1ms/step - loss: 0.1727 - acc: 0.9408 - val_loss: 0.1113 - val_acc: 0.9608
Epoch 2/4
158719/158719 [==============================] - 165s 1ms/step - loss: 0.1182 - acc: 0.9553 - val_loss: 0.0945 - val_acc: 0.9665
Epoch 3/4
158719/158719 [========

In [0]:
data = pd.DataFrame({'acc':history.history['acc'],
                    'loss': history.history['loss'],
                    'val_acc': history.history['val_acc'],
                    'val_loss': history.history['val_loss']})

In [0]:
np.array(auc).mean()

In [0]:
np.array(roc).mean()

In [0]:
X_train = pd.concat([pd.Series(X), ps_labels.loc[ps_labels['mean_score_without_5fold']>0.7, 'text']], axis=0)
y_train = pd.concat([pd.Series(y), ps_labels.loc[ps_labels['mean_score_without_5fold']>0.7, 'y']], axis=0)

tokenizer = text.Tokenizer(num_words=max_features, oov_token='unknown')
tokenizer.fit_on_texts(X_train)

list_tokenized_train = tokenizer.texts_to_sequences(X_train)
list_tokenized_test  = tokenizer.texts_to_sequences(X_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test  = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

In [29]:
model   = gru_keras(max_features, maxlen, dropout_rate, embed_dim, rec_units)

y_train = np.array(y_train)
y_test  = np.array(y_test)

print('Fitting')
model.fit(X_train, y_train,   batch_size=batch_size, epochs=4, shuffle=True, verbose=1)
probs = model.predict(X_test, batch_size=batch_size, verbose=1)
auc_f = average_precision_score(y_test, probs)
roc_f = roc_auc_score(y_test, probs)
model.save_weights(f'{models_path}/BGRU_SEMI_07_onlyTrain.h5')
del model
K.clear_session()

Fitting
Epoch 1/4
163913/163913 [==============================] - 162s 986us/step - loss: 0.1757 - acc: 0.9387
Epoch 2/4
163913/163913 [==============================] - 161s 983us/step - loss: 0.1165 - acc: 0.9567
Epoch 3/4
163913/163913 [==============================] - 161s 985us/step - loss: 0.1102 - acc: 0.9583
Epoch 4/4
64830/64830 [==============================] - 20s 304us/step


In [30]:
round(auc_f, 3)

0.856

In [31]:
round(roc_f, 3)

0.973